# 第4章 文書のランキング

In [1]:
# これまでに定義した関数の読み込み

from chapter01 import get_string_from_file
from chapter02 import get_words_from_file
from chapter03 import get_words, bows_to_cfs, load_aozora_corpus, get_bows,\
    add_to_corpus, get_weights, translate_bows, get_tfidfmodel_and_weights

In [2]:
# Listing 4.1 #

def jaccard(X, Y):
    x = set(X)
    y = set(Y)
    a = len(x.intersection(y))
    b = len(x.union(y))
    if b == 0:
        return 0
    else:
        return a/b

In [3]:
# Listing 4.2 #

# 紹介文を読み込む．i番目の書籍の紹介文はtexts[i]
book_texts = [get_string_from_file('data/ch04/%d.txt' % i) for i in range(10)]          

# 各文書の重みを計算
# 補助的なコーパスとして青空文庫を利用
tfidf_model, dic, book_weights = get_tfidfmodel_and_weights(book_texts)

# 書籍ごとに上位10 の特徴語(のID) のリストを作成
# book_weights にはID とTF・IDFの値が組になっている
# そこからID だけを抜き出す
keyword_lists = [[x[0] for x in w[:10]] for w in book_weights]

# results の要素は(i, i 番目の書籍との類似度)
#『定理のつくりかた』の番号は9(9.txt)
results = [(x, jaccard(keyword_lists[9], keyword_lists[x])) for x in range(9)]

# 類似度で降順にソート
results.sort(key=lambda x: x[1], reverse=True)

# 書籍のタイトルを読み込む．i番目の書籍のタイトルはtitles[i]
with open('data/ch04/book-titles.txt', encoding='UTF-8') as f:
    titles = f.read().strip().split('\n')

# ランキング結果を表示
for x in range(9):
    print('%s %.4f'% (titles[results[x][0]], results[x][1]))

ブラックホールと時空の方程式 0.1111
逆数学 0.1111
64の事例からわかる金属腐食の対策 0.0526
Coq/SSReflect/MathCompによる定理証明 0.0526
基礎からわかる高分子材料 0.0000
ゼロからはじめるVisual_C#入門 0.0000
実践_地域・まちづくりワーク 0.0000
応用数学問題集 0.0000
生態系生態学(第2版) 0.0000


In [4]:
# Listing 4.3 #

from gensim import corpora, models

texts = ['花より団子．とにかく団子．', 'みたらしよりあんこ']
words = [get_words(text, keep_pos=['名詞']) for text in texts]
dic = corpora.Dictionary(words)

# 辞書の中身を列挙
for i in range(len(dic)):
    print('dic[%d] = %s' % (i, dic[i]))                                              

bows = [dic.doc2bow(w) for w in words]
tfidf = models.TfidfModel(bows)

# 1番目の文書の語とその重みを取得
weights =  tfidf[bows[0]]

# 説明の都合上，小数点以下4位より下の桁を省略
weights = [(i, round(j, 4)) for i, j in weights]

# 1番目の文書の語とその重みを表示
print('weights = ',  weights)

dic[0] = 団子
dic[1] = 花
dic[2] = あんこ
dic[3] = みたらし
weights =  [(0, 0.8944), (1, 0.4472)]


In [5]:
#  Listing 4.4 #

from gensim.similarities import MatrixSimilarity

def vsm_search(texts, query):
    tfidf_model, dic, text_weights = get_tfidfmodel_and_weights(texts)

    index = MatrixSimilarity(text_weights,  num_features=len(dic))

    # queryのbag-of-wordsを作成し，重みを計算
    query_bows = get_bows([query], dic)
    query_weights = get_weights(query_bows, dic, tfidf_model)

    # 類似度計算
    sims = index[query_weights[0]]

    # 類似度で降順にソート
    return sorted(enumerate(sims), key=lambda x: x[1], reverse=True)

def get_list_from_file(file_name):
    with open(file_name, 'r', encoding='UTF-8') as f:
        return f.read().split()

In [6]:
# Listing 4.5 #

# 書籍紹介のデータを読み込む
book_texts = [get_string_from_file('data/ch04/%d.txt' % i) for i in range(0,10)]

# 書籍のタイトルを読み込む
titles = get_list_from_file('data/ch04/book-titles.txt')

# 類似度の計算とランキング
# book_texts[9]は「定理のつくりかた」．book_texts[:-1]はそれ以外
result = vsm_search(book_texts[:-1], book_texts[9])

# 文書番号をタイトルに変換して出力
for x in range(9):
    print('%s %.4f' % (titles[result[x][0]], result[x][1]))                       

逆数学 0.4204
ブラックホールと時空の方程式 0.2737
Coq/SSReflect/MathCompによる定理証明 0.1746
応用数学問題集 0.1195
64の事例からわかる金属腐食の対策 0.0660
実践_地域・まちづくりワーク 0.0381
ゼロからはじめるVisual_C#入門 0.0262
生態系生態学(第2版) 0.0259
基礎からわかる高分子材料 0.0243


In [7]:
# Listing 4.6 #

texts = [get_string_from_file('data/ch04/%d.txt' % i) for i in range(10)]
titles = get_list_from_file('data/ch04/book-titles.txt')
query = '数学'
result = vsm_search(texts, query)
for x in range(len(result)):
    print('%s %.4f' % (titles[result[x][0]], result[x][1]))

定理のつくりかた 0.6793
逆数学 0.4038
ブラックホールと時空の方程式 0.2001
Coq/SSReflect/MathCompによる定理証明 0.1457
応用数学問題集 0.0807
64の事例からわかる金属腐食の対策 0.0000
基礎からわかる高分子材料 0.0000
ゼロからはじめるVisual_C#入門 0.0000
実践_地域・まちづくりワーク 0.0000
生態系生態学(第2版) 0.0000
